## Test of cbpro library

In [1]:
with open('CoinBaseAPIKey.key', 'r') as f:
    contents = f.readlines()
cb_key = contents[0].split(':')[-1].strip()
cb_secret = contents[1].split(':')[-1].strip()

In [2]:
import cbpro

public_client = cbpro.PublicClient()

result = public_client.get_product_historic_rates('BTC-USDT', '2023-01-01', '2023-06-01', 86400)


In [10]:
import pandas as pd
df_res = pd.DataFrame(result)
df_res.columns = [ 'time_iso', 'low', 'high', 'open', 'close', 'volume' ]

In [11]:
df_res['Time'] = pd.to_datetime(df_res['time_iso'], format="%Y-%m-%d", errors = 'coerce')
df_res

,time_iso,low,high,open,close,volume,Time
0,1685577600,26611.84,27351.09,27220.14,26822.11,567.223972,NaT
1,1685491200,26841.45,27838.36,27689.23,27215.28,991.007433,NaT
2,1685404800,27552.65,28040.51,27736.03,27695.30,502.509372,NaT
3,1685318400,27530.82,28451.62,28081.19,27736.96,673.158599,NaT
4,1685232000,26775.89,28271.94,26859.82,28082.01,712.806237,NaT
...,...,...,...,...,...,...,...
147,1672876800,16755.98,16878.57,16854.64,16832.64,677.574942,NaT
148,1672790400,16653.48,16989.00,16676.28,16851.55,1537.125774,NaT
149,1672704000,16607.28,16776.17,16669.91,16675.13,877.292216,NaT
150,1672617600,16550.47,16793.73,16621.31,16674.64,560.136545,NaT
